This will be used for Coursera Capstone project

In [35]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [9]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


Business problem: 
There is a family that wants to open a family run authentic Japanese restaurant in Toronto. They are not a chain and are going for the loyal reliable customer base rather than flashy tourist attraction. They want to open a restaurant in a relatively high population density area first and somewhere where there isn’t a lot of similar competition and then where there isn’t a lot of other restaurants either considering that they think that this will increase their chances to gain customers if they aren’t spoiled for choice. They want a local supply of staples in Japanese cuisine such as fish, as these tend to go off rather quickly so ideally, they will want a nearby fresh fish market.  If there is a fresh food market in all of their top regions then amount of popular attractions will be considered as the final deciding factor.
Deciding factors: 

1.	Population density- to target loyal clientele that will come back, not tourists
2.	Other restaurants of the same type in the area
3.	Amount of all restaurants in the area

Nice to have all else being equal:

4.	Foot traffic, where in Toronto is there the highest density of most popular attractions

Data:



1.	Data from https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods
2.	Foursquare data on restaurants in the area containing keywords ‘Japanese food’
3.	Foursquare data on all restaurants in the area 
4.	Foursquare top 50 picks of locations for Toronto


In [20]:
import urllib.request
from bs4 import BeautifulSoup

In [ ]:
url = "https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
soup.prettify()

In [64]:
tables = soup.find_all("table")

In [65]:
len(tables)

10

In [86]:
A=[]
B=[]
C=[]
D=[]
E=[]
F=[]
for table in range(len(tables)):
    

    for row in tables[table].findAll('tr'):
        if len(row.findAll('td'))==13:
            A.append(row.findAll('td')[0].find(text=True))
            B.append(row.findAll('td')[1].find(text=True))
            C.append(row.findAll('td')[2].find(text=True))
            D.append(row.findAll('td')[3].find(text=True))
            E.append(row.findAll('td')[4].find(text=True))
            F.append(row.findAll('td')[5].find(text=True))


In [139]:
A= [x.strip('\n') for x in A]
B= [x.strip('\n') for x in B]
C= [x.strip('\n') for x in C]
D= [x.strip('\n') for x in D]
E= [x.strip('\n') for x in E]
F= [x.strip('\n,') for x in F]
F= [x.replace(',', '') for x in F]

In [141]:
neighbourhoods=pd.DataFrame(A,columns=['Neighbourhood'])
neighbourhoods['Population_density']=F


In [140]:
F

['20393',
 '1129',
 '4938',
 '3591',
 '6577',
 '5809',
 '2360',
 '2544',
 '1268',
 '1699',
 '7588',
 '2766',
 '421',
 '2652',
 '4106',
 '4013',
 '4336',
 '3507',
 '2629',
 '3858',
 '2249',
 '866',
 '3580',
 '13609',
 '4333',
 '4934',
 '1914',
 '2442',
 '3187',
 '43518',
 '2966',
 '3267',
 '6057',
 '3409',
 '3525',
 '6993',
 '8618',
 '6414',
 '445',
 '8217',
 '7943',
 '4017',
 '8843',
 '5369',
 '5275',
 '8006',
 '24358',
 '3102',
 '2073',
 '3831',
 '6693',
 '2808',
 '8870',
 '7556',
 '10387',
 '6998',
 '4795',
 '2377',
 '5764',
 '3331',
 '9767',
 '2270',
 '9975',
 '7696',
 '8576',
 '6931',
 '6267',
 '4737',
 '1166',
 '8760',
 '5530',
 '1794',
 '15846',
 '4462',
 '10793',
 '6639',
 '2688',
 '9228',
 '9228',
 '4327',
 '3066',
 '10434',
 '2505',
 '3426',
 '1132',
 '1618',
 '',
 '2164',
 '3397',
 '',
 '8624',
 '6714',
 '9258',
 '10389',
 '3523',
 '6414',
 '3285',
 '1178',
 '6425',
 '1828',
 '4944',
 '8761',
 '9774',
 '10231',
 '5073',
 '5003',
 '4425',
 '3860',
 '3654',
 '3959',
 '4112',
 '

In [113]:
from geopy.geocoders import Nominatim
latitudes=[]
longtitudes=[]
geolocator = Nominatim(user_agent="my_user_agent")
city ="Toronto"
for hood in neighbourhoods['Neighbourhood']:
    loc = geolocator.geocode(city+','+ hood)
    if type(loc)== type(geolocator.geocode(city+','+ 'Crescent Town')):
        latitudes.append(loc.latitude)
        longtitudes.append(loc.longitude)
    else:     
        latitudes.append(np.nan)
        longtitudes.append(np.nan)
   
        


In [115]:
neighbourhoods['latitude']=latitudes
neighbourhoods['longtitude']=longtitudes

Dropping the locations that couldn't be found

In [124]:
neighbourhoods.dropna(inplace=True)


In [26]:
CLIENT_ID = 'SZHE0DDBBDRLQ4KP55KYQBENUXEWBZYMENUFJENVHOTTI3JZ' # your Foursquare ID
CLIENT_SECRET = 'ECC1LMEA1PR3B2I1Z4TRAFE0QNEO22DRCLMVHNMEQL4XFAQA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SZHE0DDBBDRLQ4KP55KYQBENUXEWBZYMENUFJENVHOTTI3JZ
CLIENT_SECRET:ECC1LMEA1PR3B2I1Z4TRAFE0QNEO22DRCLMVHNMEQL4XFAQA


In [186]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [127]:
toronto_venues = getNearbyVenues(names=neighbourhoods['Neighbourhood'],
                                   latitudes=neighbourhoods['latitude'],
                                   longitudes=neighbourhoods['longtitude']
                                  )

Crescent Town
Governor's Bridge
Leaside
O'Connor–Parkview
Old East York
Thorncliffe Park
Alderwood
Centennial
Clairville
Eatonville
Humber Heights
Humberwood
Humber Valley Village
Islington – Six Points
Kingsview Village
Long Branch
Markland Wood
Mimico
New Toronto
Princess Gardens
Toronto 
Agincourt
Alexandra Park
Allenby
Amesbury
Armour Heights
Banbury
Bathurst Manor
Bay Street Corridor
Bayview Village
Bayview Woods – Steeles
Bedford Park
Bendale
Birch Cliff
Bloor West Village
Bracondale Hill
Branson
Bridle Path
Brockton
Cabbagetown
Caribou Park
Carleton Village
Casa Loma
Chaplin Estates
Christie Pits
Church and Wellesley
Clairlea
Cliffcrest
Cliffside
Corktown
Cricket Club
Davenport
Davisville
Deer Park
Discovery District
Distillery District
Don Mills
Don Valley Village
Dorset Park
Dovercourt Park
Downsview
Dufferin Grove
Earlscourt
East Danforth
Eglinton East
Elia (
Fashion District
Financial District
Flemingdon Park
Forest Hill
Fort York
Garden District
Glen Park
Grange Park
Graydo

In [128]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Crescent Town,43.695403,-79.293099,Dentonia Park,43.692809,-79.295533,Park
1,Crescent Town,43.695403,-79.293099,Dentonia Park Golf Course,43.695326,-79.289115,Golf Course
2,Crescent Town,43.695403,-79.293099,Gateway Newstand,43.694477,-79.289194,Convenience Store
3,Crescent Town,43.695403,-79.293099,Victoria Park Subway Station,43.694883,-79.288735,Metro Station
4,Governor's Bridge,43.689423,-79.369426,Chorley Park,43.687163,-79.370538,Park


In [155]:
neighbourhoods["Population_density"] = pd.to_numeric(neighbourhoods["Population_density"])

In [242]:
top_neigh=neighbourhoods.sort_values(by=['Population_density'], ascending=False).head(20)


In [243]:
top_neigh.reset_index(drop=True)

,Neighbourhood,Population_density
0,St. James Town,63765.0
1,Bay Street Corridor,43518.0
2,North York City Centre,37239.0
3,Church and Wellesley,24358.0
4,Crescent Town,20393.0
5,Regent Park,16230.0
6,Garden District,15846.0
7,Parkdale,13974.0
8,Alexandra Park,13609.0
9,Niagara,11862.0


In [244]:
top_neigh=pd.merge(top_neigh, toronto_venues, left_on='Neighbourhood', right_on='Neighborhood')


In [245]:
top_neigh.drop(columns=['Neighborhood'], inplace=True)

In [246]:
japanese_restaurants=top_neigh.loc[top_neigh['Venue Category']=='Japanese Restaurant']

In [248]:
top_neigh.set_index('Neighbourhood', inplace=True)

In [249]:
neighs=[]
for hood in top_neigh.index:
    if hood not in neighs and hood in list(japanese_restuarants['Neighbourhood']):
        top_neigh.drop([hood], axis=0, inplace=True)
        neighs.append(hood)


In [251]:
top_neigh.reset_index(inplace=True)

In [253]:
pd.unique(top_neigh['Neighbourhood'])

array(['St. James Town', 'North York City Centre', 'Crescent Town',
       'Regent Park', 'Niagara', 'Wallace Emerson', 'The Annex',
       'High Park North'], dtype=object)

In [258]:
pd.unique(top_neigh['Venue Category'])

array(['Caribbean Restaurant', 'Diner', 'Restaurant',
       'Italian Restaurant', 'Café', 'Market', 'Indian Restaurant',
       'Bistro', 'Bakery', 'Filipino Restaurant', 'Coffee Shop',
       'Grocery Store', 'Bar', 'Library', 'Playground', 'Hotel',
       'Pharmacy', 'Food & Drink Shop', 'Bank', 'Breakfast Spot',
       'Pizza Place', 'Beer Store', 'Intersection',
       'Mediterranean Restaurant', 'Middle Eastern Restaurant', 'Park',
       'Golf Course', 'Convenience Store', 'Metro Station', 'Pool',
       'Performing Arts Venue', 'Thai Restaurant', 'Sushi Restaurant',
       'Animal Shelter', 'Pub', 'Auto Dealership', 'Food Truck',
       'Fast Food Restaurant', 'Electronics Store', 'Yoga Studio',
       'Butcher', 'Salon / Barbershop', 'Dessert Shop',
       'Dumpling Restaurant', 'Bookstore',
       'Paper / Office Supplies Store', 'American Restaurant', 'Dog Run',
       "Women's Store", 'Accessories Store', 'Kids Store', 'Pet Store',
       'Optical Shop', 'Spa', 'Sandwich Pl

In [260]:
other_restaurants=top_neigh[top_neigh['Venue Category'].isin(['Caribbean Restaurant', 'Diner', 'Restaurant', 'Italian Restaurant', 'Indian Restaurant', 'Bistro', 'Filipino Restaurant', 'Pizza Place', 'Mediterranean Restaurant', 'Middle Eastern Restaurant','Thai Restaurant','Sushi Restaurant', 'Food Truck', 'Fast Food Restaurant', 'Dumpling Restaurant', 'American Restaurant', 'Sandwich Place','BBQ Joint','Seafood Restaurant','Burger Joint','Ramen Restaurant','Vietnamese Restaurant','Salad Place','Wings Joint','Comfort Food Restaurant','Portuguese Restaurant','Greek Restaurant','Vegetarian / Vegan Restaurant','Burrito Place','Korean Restaurant','Fried Chicken Joint','Falafel Restaurant'])]

In [264]:
top_neigh.set_index('Neighbourhood', inplace=True)

In [265]:
neighs=[]
for hood in top_neigh.index:
    if hood not in neighs and hood in list(other_restaurants['Neighbourhood']):
        top_neigh.drop([hood], axis=0, inplace=True)
        neighs.append(hood)

In [269]:
top_neigh.reset_index(inplace=True)

In [270]:
top_neigh.groupby(top_neigh['Neighbourhood']).count()

,Population_density,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,
Crescent Town,4,4,4,4,4,4,4
High Park North,6,6,6,6,6,6,6


In [271]:
top_neigh

,Neighbourhood,Population_density,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Crescent Town,20393.0,43.695403,-79.293099,Dentonia Park,43.692809,-79.295533,Park
1,Crescent Town,20393.0,43.695403,-79.293099,Dentonia Park Golf Course,43.695326,-79.289115,Golf Course
2,Crescent Town,20393.0,43.695403,-79.293099,Gateway Newstand,43.694477,-79.289194,Convenience Store
3,Crescent Town,20393.0,43.695403,-79.293099,Victoria Park Subway Station,43.694883,-79.288735,Metro Station
4,High Park North,10434.0,43.657383,-79.470961,Revina Park,43.658686,-79.473077,Baseball Field
5,High Park North,10434.0,43.657383,-79.470961,High Park Tuck Shop,43.656666,-79.466628,Convenience Store
6,High Park North,10434.0,43.657383,-79.470961,Ravina Park,43.660954,-79.472723,Park
7,High Park North,10434.0,43.657383,-79.470961,Sleep Country,43.653041,-79.470106,Mattress Store
8,High Park North,10434.0,43.657383,-79.470961,Bennett Parkette,43.653919,-79.467187,Park
9,High Park North,10434.0,43.657383,-79.470961,High Park Village Fitness,43.655577,-79.465269,Gym / Fitness Center
